# Basic run: How to run NZUpy and customise inputs

## Overview of NZUpy
NZUpy is an optimisation model for simulating the New Zealand Emissions Trading Scheme (NZ ETS). The model simulates interactions between carbon price, emissions, various supply components and the stockpile of units held by private participants.

The model largely repliacates the functionality of the Ministry for the Environment's (MfE's) NZ ETS model that operates in excel (using macros/VBA). However given NZUpy is coded in Python, it allows for greater flexibility and customisation by users who wish to enhance certain components (e.g., incorporating more sophisticated industrial allocation behaviours). 

## Some basics

NZUpy is structured around the `NZUpy` class, which contains all information about the scenario(s), time periods, and components of the model (supply, demand, stockpile dynamics, etc.).

There are four main steps to running NZUpy:

1. **Creating an instance**: Initialise the model
2. **Define the scope**: Specify years, and scenarios
3. **Configure scenarios**: Customise your configuration of inputs and parameters for each scenario
4. **Run the model**: Execute the model and analyse results

The model follows a builder pattern, which allows for flexible configuration.

## 1. Create NZUpy instance

Let's start by importing the NZUpy class and libaries we'll use in the notebook

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

As we're operating in a subfolder off the project root, we'll need to ensure we can find the NZUpy model, and its data directory containing necessary inputs and parameters.

In [2]:
# Add the project root to the path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import the NZUpy class
from model.core.base_model import NZUpy

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "01_basic_run"
os.makedirs(output_dir, exist_ok=True)

After that, we can initialise NZUpy, pointing it to our input data directory..

In [3]:
NZU = NZUpy(data_dir=data_dir)

## 2. Define the scope

NZUpy requires users to define the time period they wish to model. 

By default, the model is set up for a 2024 start year, and 2050 end year (aligned with MfE's excel-based model), and is recommended for the vast bulk of use cases.

In [ ]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

For this run, we'll keep things simple with a single scenario. 

Scenario naming has no effect on the model itself, and is used to enable distinct runs with their own configured settings. So we'll call ours 'Basic run'

In [ ]:
# Define scenarios - you can have multiple scenarios
NZU.define_scenarios(['Basic run'])

We then prime the model, which initialises dataframes housing our variables

In [ ]:
NZU.prime()

## 3. Configure the scenarios

After priming the model, we need to configure our scenario with the input data and parameters we want to use. 

NZUpy makes it easy to operate the model using central default inputs for all of its supply, demand and stockpile components, loaded in from CSV files in subfolders of `data/inputs/`.

We can use the `use_central_configs()` method to set all components to their central configurations for our scenario to get everything set-up quickly.


In [ ]:
# Configure our scenario (index 0) to use central configs for all components
NZU.use_central_configs(0)

Most users will want to customise certain inputs however, such as loading in an alternative set of auction settings, or trying out a different govt. forecast for forestry removals. 

Each of the CSV files in `data/inputs/` subfolders contains a range of alternative configurations that draw on different data sources (e.g., MfE & MPI published projections, Climate Change Commission annual estimates of the surplus, etc.)

We can use the `list_available_configs()` method to quicly pull up alternative options for each component.

In [ ]:
# Show available configurations for each component
print("Available configurations:")
print(f"Emissions configs:", NZU.list_available_configs('emissions'))
print(f"Auction configs:", NZU.list_available_configs('auction'))
print(f"Industrial configs:", NZU.list_available_configs('industrial'))
print(f"Forestry configs:", NZU.list_available_configs('forestry'))
print(f"Demand model configs:", NZU.list_available_configs('demand_model'))
print(f"Stockpile configs:", NZU.list_available_configs('stockpile'))


For this demonstration notebook, we'll look more closely at stockpile and surplus trends. 

We'll do this by swapping out NZUpy's default config (drawn from the Climate Change Commission's 2024 estimates) with a new Ernst & Young central estimate featured in a Sep 2024 report to the Ministry for the Environment. This stockpile config is listed as `EY24_central`.

_nb. Further metadata on where each config is sourced from can be found in respective CSV files in `data/inputs` subfolders._

In [ ]:
# Get stockpile data
print("~~~~~~~~| STOCKPILE & SURPLUS BEFORE EDIT |~~~~~~~~")
stockpile_data = NZU.show_inputs('stockpile', scenario_name='Basic run')

# Use the EY24_central config for our scenario (index 0)
NZU.use_config(0, 'stockpile', 'EY24_central')

print("~~~~~~~~| STOCKPILE & SURPLUS AFTER EDIT |~~~~~~~~")
# Get updated stockpile data
stockpile_data = NZU.show_inputs('stockpile', scenario_name='Basic run')


_You'll see we now have a lower starting surplus balance for our reference year (2023), down to 52.4 million NZUs from the Climate Commission's central estimate of 68.1 million..._

## 4. Run the model

Now that we've configured our scenario, we can run the model:

In [ ]:
# Run the model
results = NZU.run()

## 5. Analyse the results

Now that we've got our results in tow, we can undertake any number of analyses. We'll keep things simple here, and call a list of variables available to us, so we can pick some result variables we're interested in

In [ ]:
NZU.list_variables()

As we're dealing with pandas dataframes, we can use typical methods from numpy and pandas to get a quick sense of the results

In [ ]:
# Get carbon price data for our scenario
carbon_price = NZU.prices.xs('carbon_price', level='variable', axis=1)
print("Carbon price trajectory ($/tCO₂-e):")
print(carbon_price.head().round(2))

But in this case, given we've swapped in an alternative starting surplus estimate, we're most interested in stockpile and surplus balance, so lets take a look at these.

In [ ]:
# Get stockpile balance data
stockpile_balance = NZU.stockpile.xs('balance', level='variable', axis=1)
surplus_balance = NZU.stockpile.xs('surplus_balance', level='variable', axis=1)
non_surplus_balance = NZU.stockpile.xs('non_surplus_balance', level='variable', axis=1)

print("\nStockpile balance in 2050 (kt CO₂-e):")
print(f"Total stockpile: {stockpile_balance.iloc[-1, 0]:,.0f}")
print(f"Surplus: {surplus_balance.iloc[-1, 0]:,.0f}")
print(f"Non-surplus: {non_surplus_balance.iloc[-1, 0]:,.0f}")

These stockpile balance levels in 2050 are about the same as our balance at the start of time period (end 2023), so it would be interesting to see how these evolve over time.

Helpfully, we can call some pre-configureds chart to make our life easier...

In [ ]:
# Load chart generator
from model.utils.chart_generator import ChartGenerator

# Initialise chart generator
chart_gen = ChartGenerator(NZU)

# Generate stockpile balance chart - referring to our scenario index 0
stockpile_chart = chart_gen.stockpile_balance_chart(NZU.scenarios[0])
# We can then display the chart in the notebook
display(stockpile_chart) 
# And we can also save the chart directly as PNG
stockpile_chart.write_image(str(output_dir / "stockpile_balance.png"))

# Generate supply-demand balance chart
supply_demand_chart = chart_gen.supply_demand_balance_chart(NZU.scenarios[0])
display(supply_demand_chart)
supply_demand_chart.write_image(str(output_dir / "supply_demand_balance.png"))

With that we can see that while EY's reduced starting surplus balance sees the surplus quickly reduced to 0, the surplus begins to increase again in the late 2030s and 2040s, with supply exceeding demand (shown in the supply-demand balance chart). However, the bulk of our incrase in stockpile comes from non-surplus units, where forestry units are held for future harvest liabilities.

While there is plenty more analysis of this trend we could look at, we'll wrap-up this basic run notebook by saving these results to a CSV file.

In [ ]:
# Create a DataFrame with the key results we want to export
export_data = pd.DataFrame({
    'year': carbon_price.index,
    'carbon_price': carbon_price['Basic run'],
    'stockpile_balance': stockpile_balance['Basic run'],
    'surplus_balance': surplus_balance['Basic run'],
    'non_surplus_balance': non_surplus_balance['Basic run']
})

# Set the index to year for better readability
export_data.set_index('year', inplace=True)

# Export to CSV
export_path = output_dir / 'basic_run_results.csv'
export_data.to_csv(export_path)
print(f"Results exported to: {export_path}")